# Drone Routing Optimization Problem

## Model Initialization

In [14]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp
import csv
import numpy as np

# initialize model object
mymodel = glp.Solver('Drone Routing Problem', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Parameters

### Import distance matrix

In [15]:
# import csv distance matrix as list
with open('Orders_Data.csv', newline='') as f:
    reader = csv.reader(f)
    distance = list(reader)

# remove blanks
for i in range(len(distance)):
    distance[i] = [item for item in distance[i] if item]
distance = [item for item in distance if item]
distance = np.array(distance, dtype = 'float')

# Cost per unit of distance
truck_cost_per_dist = 10.0
drone_cost_per_dist = 3.0

# Create cost matrices for truck & drone
truck_cost = distance * truck_cost_per_dist
drone_cost = distance * drone_cost_per_dist

### Initialize Model Parameters

In [16]:
# model parameters
R = list(range(1, len(distance)-1)) # Nodes excluding origin & final 'dummy' node
N = list(range(len(distance))) # All nodes
M = len(distance) + 100 # Large number
D = list(range(3)) # Drones onboard the truck

## Decision Variables

In [19]:
# Truck path from node i to j
truck_arc = [list(range(1 + len(N) * i, 1 + len(N) * (i+1))) for i in N] 
for i in N:
    for j in N:
        truck_arc[i][j] = mymodel.IntVar(0, 1, str(i+1) + "." + str(j+1))
truck_arc

[[1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 1.10, 1.11, 1.12],
 [2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 2.10, 2.11, 2.12],
 [3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 3.10, 3.11, 3.12],
 [4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 4.10, 4.11, 4.12],
 [5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 5.10, 5.11, 5.12],
 [6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 6.10, 6.11, 6.12],
 [7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 7.10, 7.11, 7.12],
 [8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 8.10, 8.11, 8.12],
 [9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 9.10, 9.11, 9.12],
 [10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 10.10, 10.11, 10.12],
 [11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 11.10, 11.11, 11.12],
 [12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 12.10, 12.11, 12.12]]

In [21]:
# Drone path from node i to j
drone_arc = np.empty((len(N),len(N), len(D)))
drone_arc[:] = np.NaN
drone_arc.tolist()

for i in R:
    for j in R:
        for k in D:
            drone_arc[i][j][k] = mymodel.IntVar(0, 1, str(i+1) + "." + str(j+1) + "." + str(k+1))
drone_arc

TypeError: float() argument must be a string or a number, not 'Variable'

In [22]:
# create position variable (lambda / u)
pos = list(N)
for i in N:
    pos[i] = mymodel.IntVar(0, len(N), "p." + str(i+1))
pos

[p.1, p.2, p.3, p.4, p.5, p.6, p.7, p.8, p.9, p.10, p.11, p.12]

In [23]:
# Not sure what this is for?
w = list(N)
for i in R:
    w[i] = mymodel.IntVar(0, len(N), "w." + str(i+1))
w

[0, w.2, w.3, w.4, w.5, w.6, w.7, w.8, w.9, w.10, w.11, 11]

## Objective

In [26]:
# create objective function
optimal_route = mymodel.Objective()
optimal_route.SetMinimization()
for i in N:
    for j in N:
        optimal_route.SetCoefficient(truck_arc[i][j], truck_cost[i][j])
        for k in D:
            optimal_route.SetCoefficient(drone_arc[i][j][k], drone_cost[i][j])

TypeError: in method 'Objective_SetCoefficient', argument 2 of type 'operations_research::MPVariable const *const'

## Constraints

### Arc constraints

In [ ]:
# first node - 1.1 + 1.2 + 1.3 + 1.4 + 1.5 = 1
#orig_node = mymodel.Constraint(1, 1)
#for i in range(N):
#    orig_node.SetCoefficient(use_arc.loc[0][i], 1)

# last node - 1.5 + 2.5 + 3.5 + 4.5 + 5.5 = 1
#last_node = mymodel.Constraint(1, 1)
#for j in range(N):
#    last_node.SetCoefficient(use_arc.loc[j][N-1], 1)

# Can't go from and to the same node - 1.1 + 2.2 + 3.3 + 4.4 + 5.5 = 1
#same_node = mymodel.Constraint(0, 0)
#for i in range(N):
#    same_node.SetCoefficient(use_arc.loc[i][i], 1)
    
# Can't go from first to dummy (1.5)
#first_to_last = mymodel.Constraint(0, 0)
#first_to_last.SetCoefficient(use_arc[N-1][0], 1)

# Can't go from dummy to first (5.1)
#last_to_first = mymodel.Constraint(0, 0)
#last_to_first.SetCoefficient(use_arc[0][N-1], 1)

In [ ]:
# Balance - Out
# 2.1 + 2.2 + 2.3 + 2.4 + 2.5 = 1

# Balance - In
# 1.2 + 2.2 + 3.2 + 4.2 + 5.2 = 1

# Flow Out

Flow_out = list(range(N))

for i in range(0, N-1): 
    Flow_out[i] = mymodel.Constraint(1, 1)
    for j in range(N): 
        Flow_out[i].SetCoefficient(use_arc[i][j], 1)
        
# Flow In

Flow_in = list(range(N))

for i in range(1, N):
    Flow_in[i] = mymodel.Constraint(1, 1)
    for j in range(N): 
        Flow_in[i].SetCoefficient(use_arc[j][i], 1)

### Position Constraints

In [ ]:
# position 1 = 1
position_1 = mymodel.Constraint(1, 1)
position_1.SetCoefficient(pos[0], 1)

# position N = N
position_N = mymodel.Constraint(1, 1)
position_N.SetCoefficient(pos[N-1], N)

# MTZ Constraint

# u[i] - u[j] + (n-1)*x[i][j] <= (n-2)

MTZ = [list(range(1 + N * i, 1 + N * (i+1))) for i in range(N)] 

for i in range(1, N):
    for j in range(1, N):
        MTZ[i][j] = mymodel.Constraint(-mymodel.infinity(), N-2)
        MTZ[i][j].SetCoefficient(use_arc[i][j], N-1)
        MTZ[i][j].SetCoefficient(pos[i], 1)
        MTZ[i][j].SetCoefficient(pos[j], -1)

In [ ]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Optimal Value = %.2f' % shortest_route.Value())

# Display optimal solution
for i in range(N):
    for j in range(N):
        print('From ', i + 1, ' to ', j + 1, ': ', use_arc[i][j].solution_value(), sep = '')
        
#        if i == N-1 or i == j:
#            continue
#        elif j == N-1:
 #           j = 1
 #           print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '')
 #       else:
  #          print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '') 